In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [15]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, filter_first_ses  # noqa: E402, F401
import mri_data.file_manager as fm
from mri_data import utils
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [7]:
logger.remove()

In [3]:
# #! Set these variables
# work_dir_name = "choroid_resegment1"
# train_dataset_file_name = "training-dataset-desktop1.json"
# prediction_postfix = "choroid_resegment_pred"
# task_name = "infer_choroid"
# modalities = ["flair", "t1"]
# save_dir = Path("/media/smbshare/3Tpioneer_bids_predictions")

In [4]:
inference_name = "flair.t1_choroid_pineal_pituitary3_pred"

In [5]:
drive_root = fm.get_drive_root()
msmri_home = Path("/home/srs-9/Projects/ms_mri")
inference_root = drive_root / "3Tpioneer_bids_predictions"
dataroot = drive_root / "3Tpioneer_bids"
clinical_data_root = drive_root / "Secure_Data" / "Large"
project_dataroot = msmri_home / "data"

In [8]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
full_dataset = dataset_proc.dataset
dataset_proc.prepare_labels(["choroid_t1_flair", "pineal", "pituitary"], ["CH", "SRS", "ED", "DT"])
dataset = dataset_proc.dataset

inference_dataset_proc = DataSetProcesser.new_dataset(inference_root, scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
inference_dataset_proc.prepare_labels("flair.t1_choroid_pineal_pituitary3_pred")
inference_dataset = inference_dataset_proc.dataset

100%|██████████| 420/420 [00:02<00:00, 169.76it/s]


In [17]:
for inference in inference_dataset[:5]:
    flair = inference.with_root(dataroot).root / "flair.nii.gz"
    t1 = inference.with_root(dataroot).root / "t1.nii.gz"
    images = [flair, t1]
    prediction = inference.label_path
    cmd = utils.open_itksnap_workspace_cmd(images, [prediction], win=True)
    print(cmd)
    # print("itksnap", "-g", flair, "-o", t1, "-s", prediction)

itksnap -g H:/3Tpioneer_bids/sub-ms1395/ses-20180810/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1395/ses-20180810/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1395/ses-20180810/flair.t1_choroid_pineal_pituitary3_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1178/ses-20191023/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1178/ses-20191023/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1178/ses-20191023/flair.t1_choroid_pineal_pituitary3_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1235/ses-20161121/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1235/ses-20161121/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1235/ses-20161121/flair.t1_choroid_pineal_pituitary3_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1154/ses-20170412/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1154/ses-20170412/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1154/ses-20170412/flair.t1_choroid_pineal_pituitary3_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1546/ses-20180205/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1546/se